In [ ]:
import mlflow
import pandas as pd

import mlflow
import pandas as pd

aggregation_mapping = {
    "average": "Avg",
    "max": "Max",
    "common_features": "ComF",
    "wcom": "WCom",
    "topk": "TopK",
}

def get_method_name(run):
    params = run.data.params
    method = params.get('recommender_strategy', 'Unknown')
    if method == "SAE":
        aggregation = params.get('SAE_fusion_strategy', 'Unknown')
        method += "-" + aggregation_mapping.get(aggregation, aggregation)
    return method.replace('_', '-')

def generate_recommendations_table(runs):
    all_rows = []
    for run in runs:        
            group_type = run.data.params.get("group_type", "none")
            approach = get_method_name(run)

            row_key = (approach)
            metrics = {
                (group_type, "G/mean"): run.data.metrics.get("CommonItemsNDCG20/mean"),
                (group_type, "U/mean"): run.data.metrics.get("NDCG20/mean"),
                (group_type, "U/min"): run.data.metrics.get("NDCG20/min"),
                (group_type, "Pop"): run.data.metrics.get("Popularity/mean"),
            }

            all_rows.append((row_key, metrics))

    # Build DataFrame from records
    records = {}
    for key, metrics in all_rows:
        if key not in records:
            records[key] = {}
        records[key].update(metrics)

    df = pd.DataFrame.from_dict(records, orient="index")
    df.index.names = ["Approach"]

    # Sort and reindex columns by dataset then metric
    df = df.sort_index(axis=1, level=[0, 1]).sort_values(
        by=["Approach"]
    )
    
    dataset_order = ["sim", "random", "outlier"]
    metric_order = ["G/mean", "U/min", "U/mean", "Pop"]

    cols = df.columns
    cols = sorted(
        cols,
        key=lambda x: (
            dataset_order.index(x[0]) if x[0] in dataset_order else len(dataset_order),
            metric_order.index(x[1]) if x[1] in metric_order else len(metric_order)
        )
    )
    df = df[cols]

    return df.reset_index()

def highlight_top3_dark_to_light(s):
    # Colors from dark to light
    colors = ['mediumseagreen', 'lightgreen']
    
    # Get sorted unique values in descending order
    top_values = s.nlargest(2).unique()
    
    # Assign background color depending on rank
    styles = ['' for _ in s]
    for rank, value in enumerate(top_values):
        styles = [
            f'background-color: {colors[rank]}' if v == value and styles[i] == '' else styles[i]
            for i, v in enumerate(s)
        ]
    return styles

def highlight_bottom3_dark_to_light(s):
    # Colors from dark to light
    colors = ['mediumblue', 'lightblue', 'paleturquoise']
    
    # Get sorted unique values in ascending order
    bottom_values = s.nsmallest(3).unique()
    
    # Assign background color depending on rank
    styles = ['' for _ in s]
    for rank, value in enumerate(bottom_values):
        styles = [
            f'background-color: {colors[rank]}' if v == value and styles[i] == '' else styles[i]
            for i, v in enumerate(s)
        ]
    return styles

In [19]:
def format_latex(df, highlight_max_cols=None, highlight_min_cols=None, round_digits=3):
    formatted_df = df.copy()
    highlight_max_cols = highlight_max_cols or []
    highlight_min_cols = highlight_min_cols or []

    for col in df.columns:
        col_values = df[col]

        if col in highlight_max_cols:
            top_two = col_values.nlargest(2).values

            def format_cell(val):
                if val == top_two[0]:
                    return f"\\textbf{{{val:.{round_digits}f}}}"
                elif val == top_two[1]:
                    return f"\\underline{{{val:.{round_digits}f}}}"
                else:
                    return f"{val:.{round_digits}f}"

        elif col in highlight_min_cols:
            bottom_two = col_values.nsmallest(2).values

            def format_cell(val):
                if val == bottom_two[0]:
                    return f"\\textbf{{{val:.{round_digits}f}}}"
                elif val == bottom_two[1]:
                    return f"\\underline{{{val:.{round_digits}f}}}"
                else:
                    return f"{val:.{round_digits}f}"
        else:
            def format_cell(val):
                return f"{val}"

        formatted_df[col] = col_values.apply(format_cell)

    return formatted_df

highlight_max_cols = [('sim', 'G/mean'), ('sim', 'U/min'), ('sim', 'U/mean'), ('random', 'G/mean'), ('random', 'U/min'), ('random', 'U/mean'), ('outlier', 'G/mean'), ('outlier', 'U/min'), ('outlier', 'U/mean')]
highlight_min_cols = [('sim', 'Pop'), ('random', 'Pop'), ('outlier', 'Pop')]

def add_significance(df, significance):
    for i, row in df.iterrows():
        for ttype in ["sim", "random", "outlier"]:
            for metric in ["$NDCG_{com}$", "$NDCG_{min}$", "Popularity"]:
                filterr = (significance['Strategy'] == row['Approach'].values[0]) & (significance['Metric'] == metric) & (significance['Group Type'] == ttype)

                is_significant = significance[filterr]['Significant'].values and significance[filterr]['Significant'].values[0]
                if is_significant:
                    df.at[i, (ttype, metric)] = f"{df.at[i, (ttype, metric)]}^*"

# Comparing with other approaches

## Group Recommendations Results for **Similar** groups sorted by MovieLens G/mean

In [20]:
experiments = ['523100174176986081', '333391697323445885']

# Select only the desired columns for aggregation

        # runs = mlflow.search_runs(
        #     experiment_ids=[exp_id],
        #     filter_string=f"params.user_set = 'full' and params.group_set = 'test'",
        #     output_format="list"
        # )
        # for run in runs:
        #     if run.data.params.get("dataset") != dataset or run.data.params.get("group_set") != "test":
        #         continue
        
runs = mlflow.search_runs(
    experiment_ids=[experiments[0]],
    filter_string="params.user_set = 'full' and params.group_set = 'test'",
    output_format="list"
)
table = generate_recommendations_table(runs)


selected_columns = []
group_types = ["sim", "random", "outlier"]
for grouptype in group_types:
    for metric in ["G/mean", "U/mean", "U/min"]:
        selected_columns.append((grouptype, metric))
    

# ADD	0.636823	0.556950	0.699173	0.498267	0.630784	0.539845	0.691403	0.543637	0.544296	0.490845	0.668927	0.472185
# 1	ELSA	0.616997	0.531441	0.693722	0.496014	0.590809	0.494636	0.676351	0.526286	0.412367	0.362904	0.597728	0.384248
# 2	ELSA_INT	0.508731	0.452824	0.633645	0.427452	0.430111	0.370422	0.581750	0.413752	0.137320	0.141332	0.409709	0.187079
# 3	EPFuzzDA	0.633619	0.555927	0.690361	0.494423	0.622119	0.538728	0.677744	0.531106	0.502470	0.477995	0.628842	0.434729
# 4	GFAR	0.324054	0.359945	0.511067	0.377762	0.287313	0.340464	0.485106	0.377095	0.192762	0.285822	0.436503	0.294918
# 5	LMS	0.621082	0.530578	0.670587	0.479160	0.605941	0.505510	0.647094	0.508674	0.448257	0.416438	0.569069	0.398767
# 6	MPL	0.479610	0.462186	0.614641	0.443885	0.418426	0.416642	0.574198	0.449292	0.276859	0.320875	0.490138	0.318158
# 7	SAE-Avg	0.635918	0.556924	0.698164	0.496354	0.633387	0.541569	0.691110	0.546145	0.545637	0.496259	0.669215	0.479514
# 8	SAE-ComF	0.589624	0.518298	0.666416	0.466037	0.561267	0.492159	0.639928	0.487395	0.420865	0.414233	0.576588	0.387508
# 9	SAE-Max	0.624611	0.553968	0.690503	0.502176	0.615537	0.537901	0.681068	0.542535	0.538108	0.501982	0.655501	0.478014
# 10	SAE-TopK	0.638526	0.558251	0.699364	0.498347	0.638136	0.545982	0.694633	0.559547	0.563814	0.505636	0.678558	0.505121
# 11	SAE-WCom



approach_order = [
    'SAE-Avg',
    'SAE-ComF',
    'SAE-Max',
    'SAE-TopK',
    'SAE-WCom',
    'ELSA',
    'ELSA_INT',
    'ADD',
    'LMS',
    'MPL',
    'EPFuzzDA',
    'GFAR',
]

table = table.sort_values(by='Approach', key=lambda x: x.map({approach: i for i, approach in enumerate(approach_order)}))


In [21]:
table

Approach       sim                                  random            \
                G/mean     U/min    U/mean       Pop    G/mean     U/min   
7    SAE-Avg  0.635918  0.556924  0.698164  0.496354  0.633387  0.541569   
8   SAE-ComF  0.589624  0.518298  0.666416  0.466037  0.561267  0.492159   
9    SAE-Max  0.624611  0.553968  0.690503  0.502176  0.615537  0.537901   
10  SAE-TopK  0.638526  0.558251  0.699364  0.498347  0.638136  0.545982   
11  SAE-WCom  0.630216  0.551937  0.694280  0.482555  0.625290  0.534339   
1       ELSA  0.616997  0.531441  0.693722  0.496014  0.590809  0.494636   
2   ELSA_INT  0.508731  0.452824  0.633645  0.427452  0.430111  0.370422   
0        ADD  0.636823  0.556950  0.699173  0.498267  0.630784  0.539845   
5        LMS  0.621082  0.530578  0.670587  0.479160  0.605941  0.505510   
6        MPL  0.479610  0.462186  0.614641  0.443885  0.418426  0.416642   
3   EPFuzzDA  0.633619  0.555927  0.690361  0.494423  0.622119  0.538728   
4       GFAR  0.561466  0.516006  0.653628  0.465758  0.513304  0.485503   

                         outlier                                
      U/mean       Pop    G/mean     U/min    U/mean       Pop  
7   0.691110  0.546145  0.545637  0.496259  0.669215  0.479514  
8   0.639928  0.487395  0.420865  0.414233  0.576588  0.387508  
9   0.681068  0.542535  0.538108  0.501982  0.655501  0.478014  
10  0.694633  0.559547  0.563814  0.505636  0.678558  0.505121  
11  0.685889  0.540571  0.541927  0.482518  0.667009  0.485838  
1   0.676351  0.526286  0.412367  0.362904  0.597728  0.384248  
2   0.581750  0.413752  0.137320  0.141332  0.409709  0.187079  
0   0.691403  0.543637  0.544296  0.490845  0.668927  0.472185  
5   0.647094  0.508674  0.448257  0.416438  0.569069  0.398767  
6   0.574198  0.449292  0.276859  0.320875  0.490138  0.318158  
3   0.677744  0.531106  0.502470  0.477995  0.628842  0.434729  
4   0.622718  0.481629  0.380442  0.421888  0.562638  0.389112

In [5]:
format_latex(
    table,
    highlight_max_cols=highlight_max_cols,
    highlight_min_cols=highlight_min_cols,
    round_digits=3
).to_latex(
    "sae_table.tex",
    index=False,
    float_format="%.3f",
    bold_rows=False,
    column_format="l|rrrr|rrrr|rrrr",
    escape=False,
    caption = (
        "Table showing the performance of different recommendation approaches on the MovieLens dataset."
        "'G/mean' shows the percentage change in mean NDCG@20 using ground-truth recommendations seen by all group members. "
        "'U/min' shows the change in the mean of the minimum NDCG@20 across group members. "
        "'U/mean' shows the change in the mean of the average NDCG@20 across group members. "
        "'Pop' shows the change in the mean popularity of recommended items."
    ),
    label="tab:other:movielens"
)

## Group Recommendations Results for **Random** groups sorted by MovieLens G/mean

In [6]:
experiments = ['523100174176986081', '333391697323445885']

# Select only the desired columns for aggregation


table = generate_recommendations_table(experiments, dataset="LastFM1k")


selected_columns = []
group_types = ["sim", "random", "outlier"]
for grouptype in group_types:
    for metric in ["G/mean", "U/mean", "U/min"]:
        selected_columns.append((grouptype, metric))
    


table = table.sort_values(
    by=["Approach"],
    key=lambda x: x.map({k: i for i, k in enumerate(approach_order)})
)

In [7]:
table

Approach       sim                                  random            \
                G/mean     U/min    U/mean       Pop    G/mean     U/min   
7    SAE-Avg  0.604748  0.646866  0.812199  0.571371  0.526981  0.561172   
8   SAE-ComF  0.586419  0.633902  0.802513  0.574658  0.505470  0.543681   
9    SAE-Max  0.605060  0.650377  0.812414  0.568531  0.526611  0.567106   
10  SAE-TopK  0.600660  0.643067  0.810358  0.561695  0.516954  0.556690   
11  SAE-WCom  0.594340  0.640112  0.807021  0.566566  0.514603  0.551968   
1       ELSA  0.559589  0.599311  0.794584  0.563900  0.434487  0.466570   
2   ELSA_INT  0.472372  0.527712  0.749238  0.486644  0.309560  0.351358   
0        ADD  0.590983  0.637572  0.805566  0.566723  0.502466  0.543923   
5        LMS  0.577049  0.621372  0.790712  0.552506  0.459305  0.503991   
6        MPL  0.473250  0.568259  0.741360  0.524369  0.318427  0.450045   
3   EPFuzzDA  0.586427  0.638022  0.801012  0.565321  0.488637  0.545399   
4       GFAR  0.520890  0.608176  0.766283  0.541165  0.365618  0.500140   

                         outlier                                
      U/mean       Pop    G/mean     U/min    U/mean       Pop  
7   0.761228  0.639047  0.450044  0.487966  0.728899  0.659697  
8   0.750370  0.667283  0.428451  0.472146  0.715132  0.648728  
9   0.760527  0.629507  0.454366  0.509125  0.725053  0.628291  
10  0.755531  0.635894  0.408902  0.447015  0.715099  0.584313  
11  0.755307  0.657513  0.439678  0.471275  0.726523  0.667992  
1   0.719669  0.607683  0.299767  0.327160  0.656873  0.543721  
2   0.634906  0.482115  0.159893  0.196496  0.540327  0.378729  
0   0.749105  0.630717  0.399799  0.442371  0.709889  0.586927  
5   0.701650  0.589517  0.345587  0.398366  0.622044  0.520673  
6   0.634949  0.515905  0.190164  0.367851  0.563757  0.434947  
3   0.731216  0.610561  0.385849  0.469195  0.672803  0.557732  
4   0.664884  0.541685  0.235461  0.427465  0.601391  0.471044

In [8]:
format_latex(
    table,
    highlight_max_cols=highlight_max_cols,
    highlight_min_cols=highlight_min_cols,
    round_digits=3
).to_latex(
    "sae_table.tex",
    index=False,
    float_format="%.3f",
    bold_rows=False,
    column_format="l|rrrr|rrrr|rrrr",
    escape=False,
    caption = (
        "Table showing the performance of different recommendation approaches on the LastFM1k dataset."
        "'G/mean' shows the percentage change in mean NDCG@20 using ground-truth recommendations seen by all group members. "
        "'U/min' shows the change in the mean of the minimum NDCG@20 across group members. "
        "'U/mean' shows the change in the mean of the average NDCG@20 across group members. "
        "'Pop' shows the change in the mean popularity of recommended items."
    ),
    label="tab:other:lastfm"
)

In [9]:
experiment_ids = ['523100174176986081', '333391697323445885']

df = generate_recommendations_table_with_best(experiment_ids, group_type="random").sort_values(by=[("MovieLens", "G/mean")], ascending=False)
df

NameError: name 'generate_recommendations_table_with_best' is not defined

## Group Recommendations Results for **divergent** groups sorted by MovieLens G/mean

In [ ]:
experiment_ids = ['523100174176986081', '333391697323445885']

df = generate_recommendations_table_with_best(experiment_ids, group_type="outlier").sort_values(by=[("MovieLens", "G/mean")], ascending=False)
df

index  Approach      Aggregation LastFM1k                      MovieLens  \
                                      G/mean    Pop U/mean  U/min    G/mean   
0      0       SAE             topk    0.409  0.584  0.715  0.447     0.564   
1      1       SAE          average    0.450  0.660  0.729  0.488     0.546   
2      2       ADD             None    0.400  0.587  0.710  0.442     0.544   
3      3       SAE             wcom    0.440  0.668  0.727  0.471     0.542   
4      4       SAE              max    0.454  0.628  0.725  0.509     0.538   
5      5  EPFuzzDA             None    0.386  0.558  0.673  0.469     0.502   
6      6       LMS             None    0.346  0.521  0.622  0.398     0.448   
7      7       SAE  common_features    0.428  0.649  0.715  0.472     0.421   
8      8      ELSA          average    0.300  0.544  0.657  0.327     0.412   
9      9       MPL             None    0.190  0.435  0.564  0.368     0.277   
10    10      GFAR             None    0.155  0.396  0.529  0.350     0.193   
11    11  ELSA_INT          average    0.160  0.379  0.540  0.196     0.137   

                         
      Pop U/mean  U/min  
0   0.505  0.679  0.506  
1   0.480  0.669  0.496  
2   0.472  0.669  0.491  
3   0.486  0.667  0.483  
4   0.478  0.656  0.502  
5   0.435  0.629  0.478  
6   0.399  0.569  0.416  
7   0.388  0.577  0.414  
8   0.384  0.598  0.363  
9   0.318  0.490  0.321  
10  0.295  0.437  0.286  
11  0.187  0.410  0.141

In [12]:
import mlflow

runs = mlflow.search_runs(
    experiment_ids=["523100174176986081"],
    filter_string="params.user_set = 'full' and params.group_set = 'test'",
    output_format="list"
)

len(runs)

36

In [13]:
runs = mlflow.search_runs(
    experiment_ids=["333391697323445885"],
    filter_string="params.user_set = 'full' and params.group_set = 'test'",
    output_format="list"
)

len(runs)

36